In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define our coupled variables to integrate

In [ ]:
def dydx(x,y):
    y_derivs = np.zeros(2)
    y_derivs[0] = y[1]
    y_derivs[1] = -1*y[0]
    return y_derivs

## Define the 5th order Cash-Karp RK method

In [ ]:
def rk4_mv_core(dydx,x_i,y_i,nv,h):
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    ytemp = np.zeros(nv)
    xtemp = np.zeros(nv)
    
    y_derivs = dydx(x_i,y_i)
    k1[:] = h*y_derivs[:]
    
    xtemp = x_i + (1/5)*h
    ytemp[:] = y_i[:] + (1/5)*k1[:]
    y_derivs = dydx(xtemp,ytemp)
    k2[:] = h*y_derivs[:]
    
    xtemp = x_i + (3/10)*h
    ytemp[:] = y_i[:] + (3/40)*k1[:] + (9/40)*k2[:]
    y_derivs = dydx(xtemp,ytemp)
    k3[:] = h*y_derivs[:]
    
    xtemp = x_i + (3/5)*h
    ytemp[:] = y_i[:] + (3/10)*k1[:] + (-9/10)*k2[:] + (6/5)*k3[:]
    y_derivs = dydx(xtemp,ytemp)
    k4[:] = h*y_derivs[:]
    
    xtemp = x_i + h
    ytemp[:] = y_i[:] + (-11/54)*k1[:] + (5/2)*k2[:] + (-70/27)*k3[:] + (35/27)*k4[:]
    y_derivs = dydx(xtemp,ytemp)
    k5[:] = h*y_derivs[:]
    
    xtemp = x_i + (7/8)*h
    ytemp[:] = y_i[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:]
    y_derivs = dydx(xtemp,ytemp)
    k6[:] = h*y_derivs[:]
    
    yipo = y_i + (37/378)*k1 + 0*k2 + (250/621)*k3 + (125/594)*k4 + 0*k5 + (512/1771)*k6
    return yipo

## Define an adaptive step size driver for RK5

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    imax= 10000
    i =0
    Delta = np.full(nv, 2*tol)
    h_step = h

    while(Delta.max()/tol>1.0):
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        Delta = np.fabs(y_2 - y_11)
        
        if(Delta.max()/tol>1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad")
            raise StopIteration("Ending after i = ",i)
            
        i += 1
        
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    return y_2, h_new, h_step

## Define a wrapper for RK5

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    xi = a
    yi = y_a.copy()
    h = 1.0e-4*(b-a)
    imax = 10000
    i = 0
    
    nv = len(y_a)
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    flag = 1

    while(flag):
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        h = h_new
        
        if(xi+h_step>b):
            h = b-xi
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            flag = 0
            
        xi += h_step
        yi[:] = yi_new[:]

        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new

        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        i += 1

        s = "i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,h_step,b)
        print(s)
        
        if(xi==b):
            flag = 0
    
    return x,y

## Perform the Integration

In [ ]:
a = 0.0
b = 2.0*np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
mv = 2
tolerance = 1.0e-6

x,y = rk4_mv(dydx,a,b,y_0,tolerance)

## Plot the result

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(x,y[:,0],'go',label='y(x)')
plt.plot(x,y[:,1],'ro',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),'b',label='sin(x)')
plt.plot(xx,np.cos(xx),'k',label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False,prop={'size':12})

## Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error=(y[:,1]-cosine)

plt.figure(figsize=(8,5))
plt.plot(x,y_error,'b',label="y(x) Error")
plt.plot(x,dydx_error,'r',label="dydx(x) Error")
plt.legend(loc='lower left',frameon=False,prop={'size':12})